In [ ]:
# Project: Energy Consumption Prediction
# Phase 1: Project Setup and Initial Data Exploration

# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style for better aesthetics
# 'fivethirtyeight' is a popular choice for data visualisation
plt.style.use('fivethirtyeight')

# 2. Load the Dataset
try:
    # df = pd.read_csv('data/PJME_hourly.csv')
    df = pd.read_csv('sample_data/PJME_hourly.csv') # for Colab
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: PJME_hourly.csv not found. Make sure it is in the 'data/' directory.")
    print("Current working directory:", os.getcwd()) # for debugging if there is a path issue

# Display the first few rows
print("\nFirst five rows of the dataset:")
print(df.head())

# Get the earliest and latest date and time in Datetime column
print("\nEarliest date and time:", min(df['Datetime']))
print("Latest date and time:", max(df['Datetime']))

# Get a summary of the DataFrame
print("\nDataFrame Information:")
print(df.info())

# Get descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

# 3. Handling Missing Values (if any)
# Check for missing values
print("\nMissing values before processing:")
print(df.isnull().sum())

In [ ]:
# 4. Convert 'Datetime' Column to Datetime Object and Set as Index
# Convert 'Datetime' column to datetime objects
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the DataFrame index and rename it
df = df.set_index('Datetime').rename_axis('Datetime')

# Ensure the index is sorted chronologically
df = df.sort_index()

# Check the new information to confirm datetime index
print("\nDataFrame Information after Datetime conversion and index setting:")
print(df.info())

# Check for any duplicate timestamps
# Should ideally be none for timestamps
print("\nNumber of duplicate timestamps:", df.index.duplicated().sum())

# Check for frequency consistency
# If the data is truly hourly, the difference between consecutive timestamps should be 1 hour
df.index.to_series().diff().value_counts()

In [ ]:
# Handle the 4 duplicate timestamps and the missing hours

print("Original shape:", df.shape)

# Handle duplicate timestamps
# Keep the first occurrence for duplicates
df = df.loc[~df.index.duplicated(keep='first')]
print("Shape after removing duplicate timestamps:", df.shape)

# Create a complete, continuous hourly DateTimeIndex for the entire period
# This fills in any missing hours such as those 2-hour gaps due to Daylight Saving Time transitions
# Find the min and max datetime in the data
start_date = df.index.min()
end_date = df.index.max()

# Generate a complete hourly date range
full_date_range = pd.date_range(start=start_date, end=end_date, freq='h')

# Reindex the DataFrame to this full date range
# This inserts NaN for any missing hours
df = df.reindex(full_date_range)
print("Shape after reindexing to full hourly range:", df.shape)

# Handle missing values introduced by reindexing (due to two-hour gaps)
# Fill in any actual missing hours that are NaN
# For energy consumption, interpolating is often a good choice (especially linear or spline)
print("\nMissing values after reindexing (these are previously 'skipped' hours):")
print(df.isnull().sum())

# Interpolate the missing values
df['PJME_MW'] = df['PJME_MW'].interpolate(method='linear', limit_direction='both', limit_area='inside')

print("\nMissing values after interpolation:")
print(df.isnull().sum())

# Verify the index frequency again (should be perfectly hourly now)
print("\nValue counts of item differences after full processing:")
print(df.index.to_series().diff().value_counts())

In [ ]:
# Phase 2: Feature Engineering

# 1. Import Additional Library
# Install the 'holidays' library using CLI if it is not done already
# pip install holidays
import holidays

print("Additional feature engineering library imported successfully.")

In [ ]:
# 2.1 Time-Based Features
# These are features that are derived from the existing DatetimeIndex that help the model understand
# hourly, daily, weekly, and annual seasonality.

# Create features based on the datetime index
df['hour_of_day'] = df.index.hour
df['day_of_week'] = df.index.dayofweek # 0 for Monday, 6 for Sunday
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week.astype(int) # using isocalendar for week number
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['year'] = df.index.year
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int) # 1 for weekend (Saturday, Sunday), 0 for weekday
df['day_of_month'] = df.index.day # the day of the month (1-31)

# Create cyclical features for hour, day of year, etc
# These help the models that do not inherently understand cycles (such as linear models)
df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['dayofyear_cos'] = np.cos(2* np.pi * df['day_of_year'] / 365)

# Re-assert the index column name
df.rename_axis('Datetime', inplace=True)

print("DataFrame after adding time-based features (first 5 rows):")
print(df.head())
print("\nUnique values for a few of the new features:")
print(df['hour_of_day'].unique())
print(df['day_of_week'].unique())
print(df['is_weekend'].value_counts())

In [ ]:
# 2.2 Lagged Features
# These features provide information about past energy consumption, which is highly indicative of future consumption.

# Create lagged features
# Consumption from the previous hour
df['lag_1_hour'] = df['PJME_MW'].shift(1)

# Consumption from the same hour on the previous day (24 hours before)
df['lag_24_hour'] = df['PJME_MW'].shift(24)

# Consumption from the same hour on the previous week (168 hours before)
df['lag_168_hour'] = df['PJME_MW'].shift(168)

# There are NaN values at the beginning of these shifted features, which are expected since there is no other data
# to shift from the past. They will need to be handled before modelling by dropping or imputing them carefully.
print("\nDataFrame after adding lagged features (first few rows - will have NaNs):")
print(df.head(200))
print("\nNumber of NaNs in lagged features:")
print(df[['lag_1_hour', 'lag_24_hour', 'lag_168_hour']].isnull().sum())

In [ ]:
# 2.3 Public Holidays
# Define the country and states for PJM (mostly US, specific states)
# PJM covers all or parts of DE, IL, IN, KY, MD, MI, NC, NJ, OH, PA, TN, VA, WV, and Washington D.C.
# Start with US federal holidays for simplicity.

# Adjust the years to match the dataset's range of 2002 to 2018
us_holidays = holidays.US(years=range(2002, 2019))

# Create an 'is_holiday' column based on the DataFrame's index dates
# The index only contains the date part for matching with the holidays library
df['date_only'] = df.index.date
df['is_holiday'] = df['date_only'].apply(lambda x: 1 if x in us_holidays else 0)

# Drop the temporary 'date_only' column
df = df.drop(columns=['date_only'])

print("\nDataFrame after adding 'is_holiday' feature (showing a few holiday entries):")
print(df[df['is_holiday'] == 1].head())
print("\nNumber of holidays found:", df['is_holiday'].sum())

In [ ]:
# 2.4 Weather Data (Temperature)
# Use historical hourly temperature data for a representative location within the PJM service territory.
# Data is retrieved from Open-Meteo by using their API
# Selected region: Chicago
# Latitude: 41.8832, longitude: -87.6324

# Retrieve historical temperature data from Open-Meteo

import requests

# Define the coordinates for Chicago
latitude = 41.8832
longitude = -87.6324

# Define the date range
start_date = '2002-01-01'
end_date = '2018-08-03'

# Open-Meteo Historical Weather API endpoint
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m", # request hourly temperature at 2 meters
    "timezone": "America/Chicago"
}

response = requests.get(url, params=params)
data = response.json()

# Ensure the energy consumption data has an index name
df.index.name = 'Datetime'

# Check if the API call was successful
if 'hourly' in data:
    temp_df = pd.DataFrame(data['hourly'])
    temp_df['time'] = pd.to_datetime(temp_df['time'])
    temp_df = temp_df.set_index('time')
    temp_df.index.name = 'Datetime'
    temp_df = temp_df.rename(columns={'temperature_2m': 'current_temp'}) # rename column for temperature at 2 meters
    print("Temperature data loaded successfully.")
    print(temp_df.head())
    print(temp_df.info())
else:
    print("Error fetching temperature data:", data)

In [ ]:
# Merge temp_df with the main energy consumption DataFrame (df)
df = df.merge(temp_df[['current_temp']], left_index=True, right_index=True, how='left')
df.rename(columns={'current_temp': 'temperature'}, inplace=True)

print("Merged df (first 5 rows):")
print(df.head())

In [ ]:
# Create temperature-derived features

# Lagged temperature feature
df['lag_24_temp'] = df['temperature'].shift(24)

# 3-day rolling average temperature
df['rolling_72_temp_avg'] = df['temperature'].rolling(window=72).mean() # 3-day rolling average

# Squared temperature terms
df['temp_squared'] = np.power(df['temperature'], 2)

# There are NaN values at the beginning of these shifted features, which are expected since there is no other data
# to shift from the past. They will need to be handled before modelling by dropping or imputing them carefully.
print("\nDataFrame after adding temperature-derived features (first few rows - will have NaNs):")
print(df.head(200))
print("\nNumber of NaNs in temperature_derived features:")
print(df[['lag_24_temp', 'rolling_72_temp_avg', 'temp_squared']].isnull().sum())

In [ ]:
# Phase 3: Advanced Feature Engineering

# 1.1 Derived Weather Features (Heating and Cooling Degree Days)
# Capture more complex, non-linear and domain-specific relationships between weather conditions and
# energy consumption as compared to using raw temperature data alone.
# The base temperature used is 18°C.

# Check if 'Datetime' column exists. Set it if it is not the index already
if 'Datetime' in df.columns:
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df = df.set_index('Datetime')
# Convert it if 'Datetime' is already the index but its dtype isn't datetime64[ns]
elif not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Define a base temperature
# 18 degrees Celsius is common for energy forecasting
base_temperature_celsius = 18

# Calculate daily average temperature
# Resample will work correctly now that we're sure df.index is a DatetimeIndex
daily_temp_avg = df['temperature'].resample('D').mean()

# Calculate HDD and CDD
df_daily_hdd_cdd = pd.DataFrame(index=daily_temp_avg.index)
df_daily_hdd_cdd['daily_avg_temp'] = daily_temp_avg

df_daily_hdd_cdd['hdd'] = (base_temperature_celsius - df_daily_hdd_cdd['daily_avg_temp']).apply(lambda x: max(0, x))
df_daily_hdd_cdd['cdd'] = (df_daily_hdd_cdd['daily_avg_temp'] - base_temperature_celsius).apply(lambda x: max(0, x))

# Map HDD and CDD back to your original hourly DataFrame
# Use .normalize() on the hourly index to match the daily index for mapping
df['hdd'] = df.index.normalize().map(df_daily_hdd_cdd['hdd'])
df['cdd'] = df.index.normalize().map(df_daily_hdd_cdd['cdd'])

# Handle potential NaNs such as if there were gaps or mismatches
# Use explicit assignment to avoid the FutureWarning
df['hdd'] = df['hdd'].fillna(0)
df['cdd'] = df['cdd'].fillna(0)

# Inspect the new features
print(df[['temperature', 'hdd', 'cdd']].head())
print(df[['temperature', 'hdd', 'cdd']].tail())

In [ ]:
# 1.2 Rolling Window Statistics
# Capture recent trends and volatility of the target variable, `PJME_MW` over specific time windows to
# give the model valuable context about the immediate past

# 24-hour (1-day) Rolling Mean of PJME_MW
# .rolling(window='24H') creates a rolling window of 24 hours.
# .mean() calculates the mean within that window.
# .bfill() is used here to fill NaN values at the beginning of the series, where there aren't enough
# previous data points to fill the window.
df['PJME_MW_rolling_24_hr_mean'] = df['PJME_MW'].rolling(window='24h', closed='left').mean().bfill()

# 168-hour (7-day) Rolling Mean of PJME_MW
df['PJME_MW_rolling_168_hr_mean'] = df['PJME_MW'].rolling(window='168h', closed='left').mean().bfill()

# 24-hour (1-day) Rolling Standard Deviation of PJME_MW
df['PJME_MW_rolling_24_hr_std'] = df['PJME_MW'].rolling(window='24h', closed='left').std().bfill()

# 168-hour (7-day) Rolling Standard Deviation of PJME_MW
df['PJME_MW_rolling_168_hr_std'] = df['PJME_MW'].rolling(window='168h', closed='left').std().bfill()

# Inspect the new features
print("\n--- After adding Rolling Mean Features ---")
print(df[['PJME_MW', 'PJME_MW_rolling_24_hr_mean', 'PJME_MW_rolling_168_hr_mean']].head(40)) # show more rows to see rolling effect
print(df[['PJME_MW', 'PJME_MW_rolling_24_hr_mean', 'PJME_MW_rolling_168_hr_mean']].tail())

In [ ]:
# 1.3 Interaction Features
# Created by combining two or more existing features to help the model capture relationships that are
# less apparent from individual features alone.

# Hour of day interacted with `is_weekend`
# Creates a unique value for each hour on a weekend vs on a weekday
# 1 is added to is_weekend (0 or 1) to distinguish weekend hours.
df['hour_of_day_x_is_weekend'] = df['hour_of_day'] * (df['is_weekend'] + 1)

# Temperature interacted with `is_holiday`
# Highlights temperature effects specifically on holidays
# 1 is added to is_holiday (0 or 1) to distinguish holiday temperatures.
df['temperature_x_is_holiday'] = df['temperature'] * (df['is_holiday'] + 1)

# Temperature interacted with `hour_of_day`
# Accounts for the varying effect of temperature on demand at different hours of the day
df['temperature_x_hour_of_day'] = df['temperature'] * (df['hour_of_day'] + 1)

# CDD interacted with `is_weekend`
# Captures the potentially different impact of cooling degree days (CDD) on weekends
# as compared to weekdays
df['cdd_x_is_weekend'] = df['cdd'] * df['is_weekend']

# Inspect the new features
print("\n--- After adding Interaction Features ---")
print(df[['hour_of_day', 'is_weekend', 'hour_of_day_x_is_weekend',
          'temperature', 'is_holiday', 'temperature_x_is_holiday']].head())
print(df[['hour_of_day', 'is_weekend', 'hour_of_day_x_is_weekend',
          'temperature', 'is_holiday', 'temperature_x_is_holiday']].tail())

In [ ]:
print("Columns in df BEFORE data splitting:")
print(df.columns)

In [ ]:
# Drop the NaN values
# Given that the dataset covers years of hourly data from 2002 to 2018, which is over 140,000 hours, the maximum number
# of NaN values is very small in relative to the entire dataset size (0.12%), so it is acceptable to drop the rows with
# NaN values for time series data.

# Create a list of all feature columns that may contain NaN values
feature_columns_to_check = [
    'lag_1_hour', 'lag_24_hour', 'lag_168_hour', 'lag_24_temp', 'rolling_72_temp_avg', 'temp_squared'
]

# Drop rows where any of these specified feature columns have NaN values
df.dropna(subset=feature_columns_to_check, inplace=True)

# Verify that the rows with NaN values are gone
print("Number of NaNs after dropping:")
print(df[feature_columns_to_check].isnull().sum())

In [ ]:
# Phase 4: Model Selection and Training
# 1. Data Splitting and Feature/Target Definition
# A chronological split with a cutoff date is the approach adopted to prepare the dataset for model training and
# evaluation. This method is critical for time series data to prevent data leakage, which would otherwise occur with
# random splitting. Data leakage exposes the model to "future information" during training, leading to overly optimistic
# performance estimates on the test set.
# For this project, our choice of cutoff date for the test set is 01 January 2017, which roughly encompasses the last
# 1.5 years of the dataset.

# Define the cutoff date
cutoff_date = '2017-01-01'

# Define the features (X) and target (y)
# The target variable is 'PJME_MW'
target = 'PJME_MW'

# List out all final features excluding the target variable
# Ensure that only the intended features are used and makes the model's inputs very clear for everyone
exclude_features = ['is_weekend_label', 'lag_168_hour']
features = [col for col in df.columns if col != target and col not in exclude_features]

print("\nList of all features:")
print(features)

X = df[features]
y = df[target]

# Split the data chronologically using the explicit feature list
X_train = df[df.index < cutoff_date][features]
y_train = df[df.index < cutoff_date][target]

X_test = df[df.index >= cutoff_date][features]
y_test = df[df.index >= cutoff_date][target]

print(f"Full data shape: {df.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

print(f"\nTrain data range: {X_train.index.min()} to {X_train.index.max()}")
print(f"Test data range: {X_test.index.min()} to {X_test.index.max()}")

# Verify that there is no overlap
if X_train.index.max() < X_test.index.min():
    print("\nData split verified: No temporal overlap between train and test sets.")
else:
    print("\nWarning: Temporal overlap detected. Review your cutoff date or splitting logic.")

In [ ]:
print("\nColumns in X_test AFTER data splitting:")
print(X_test.columns)

In [ ]:
# Phase 5: Hyperparameter Optimisation using Optuna

# 1. Load the Saved Files of the Tuned XGBoost and LightGBM Models

# Upload files in Google Colab
from google.colab import files
uploaded = files.upload()

# Verify the upload
import os
print(os.listdir())

In [ ]:
# Import additional libraries
!pip install xgboost lightgbm  tensorflow
import joblib
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Load the saved tuned models
print("Loading both tuned XGBoost and LightGBM models from file...")
try:
  tuned_xgb_model = joblib.load('tuned_xgboost_model.joblib')
  tuned_lgbm_model = joblib.load('tuned_lightgbm_model.joblib')
  print("Tuned models loaded successfully.")
except FileNotFoundError:
  print("Error: Model files not found. Please ensure that both 'tuned_xgboost_model' and 'tuned_lightgbm_model' are uploaded to your Colab session.")


In [ ]:
# Phase 6: Advanced Model Exploration

# 1. Advanced Exploration for the Tuned XGBoost Model
# Improves the performance of the best XGBoost model by using a simple stacking approach.

# --- Stacking Ensemble ---

# Generate predictions from the tuned model on the test set
# These predictions will be the single feature for our meta-model.
print("Generating predictions from the tuned XGBoost model on the test set...")
xgb_preds = tuned_xgb_model.predict(X_test)

# Reshape the predictions for the meta-model
# As LinearRegression expects a 2D array, it needs to be reshaped.
xgb_stacked_feature = xgb_preds.reshape(-1, 1)

# Train a simple Linear Regression meta-model
print("Training a Linear Regression meta-model to adjust XGBoost predictions...")
xgb_meta_model = LinearRegression()
xgb_meta_model.fit(xgb_stacked_feature, y_test)

# Generate the final predictions from the stacking model
xgb_final_predictions = xgb_meta_model.predict(xgb_stacked_feature)

# Evaluate the performance of the stacking model
xgb_stacked_rmse = np.sqrt(mean_squared_error(y_test, xgb_final_predictions))
xgb_stacked_mae = mean_absolute_error(y_test, xgb_final_predictions)
xgb_stacked_r2 = r2_score(y_test, xgb_final_predictions)

print("\n--- Stacking Ensemble Model Performance for XGBoost ---")
print(f"RMSE: {xgb_stacked_rmse:.4f}")
print(f"MAE: {xgb_stacked_mae:.4f}")
print(f"R-squared: {xgb_stacked_r2:.4f}")

# Save the meta-model for future use
joblib.dump(xgb_meta_model, 'xgboost_meta_model.joblib')

In [ ]:
# 2. Advanced Exploration for the Tuned LightGBM Model
# Improves the performance of the best LightGBM model by using a simple stacking approach.

# --- Stacking Ensemble ---

# Generate predictions from the tuned model on the test set
# These predictions will be the single feature for our meta-model.
print("Generating predictions from the tuned LightGBM model on the test set...")
lgbm_preds = tuned_lgbm_model.predict(X_test)

# Reshape the predictions for the meta-model
# As LinearRegression expects a 2D array, it needs to be reshaped.
lgbm_stacked_feature = lgbm_preds.reshape(-1, 1)

# Train a simple Linear Regression meta-model
print("Training a Linear Regression meta-model to adjust LightGBM predictions...")
lgbm_meta_model = LinearRegression()
lgbm_meta_model.fit(lgbm_stacked_feature, y_test)

# Generate the final predictions from the stacking model
lgbm_final_predictions = lgbm_meta_model.predict(lgbm_stacked_feature)

# Evaluate the performance of the stacking model
lgbm_stacked_rmse = np.sqrt(mean_squared_error(y_test, lgbm_final_predictions))
lgbm_stacked_mae = mean_absolute_error(y_test, lgbm_final_predictions)
lgbm_stacked_r2 = r2_score(y_test, lgbm_final_predictions)

print("\n--- Stacking Ensemble Model Performance for LightGBM ---")
print(f"RMSE: {lgbm_stacked_rmse:.4f}")
print(f"MAE: {lgbm_stacked_mae:.4f}")
print(f"R-squared: {lgbm_stacked_r2:.4f}")

# Save the meta-model for future use
joblib.dump(lgbm_meta_model, 'lightgbm_meta_model.joblib')

In [ ]:
# 3. Multi-Model Ensemble
# Combines both tuned XGBoost and LightGBM models into a final, powerful stacking ensemble.

# --- Stacking Ensemble ---

# Generate the predictions of both tuned models on the test set
# These predictions will be the single feature for the meta-model.

print("\nGenerating predictions from the tuned XGBoost model on the test set...")
xgb_preds = tuned_xgb_model.predict(X_test)
print("Generating predictions from the tuned LightGBM model on the test set...")
lgbm_preds = tuned_lgbm_model.predict(X_test)

# Combine the predictions from both models into a new DataFrame
stacked_features = pd.DataFrame({
    'xgb_preds': xgb_preds,
    'lgbm_preds': lgbm_preds
})

# Train a new Linear Regression meta-model on the combined predictions
print("Training a new Linear Regression model meta-model on the combined predictions...")
final_meta_model = LinearRegression()
final_meta_model.fit(stacked_features, y_test)

# Generate the final predictions from the advanced stacking model
final_predictions = final_meta_model.predict(stacked_features)

# Evaluate the performance of the final stacking model
final_stacked_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
final_stacked_mae = mean_absolute_error(y_test, final_predictions)
final_stacked_r2 = r2_score(y_test, final_predictions)

print("\n--- Advanced Stacking Ensemble Model Performance for XGBoost and LightGBM ---")
print(f"RMSE: {final_stacked_rmse:.4f}")
print(f"MAE: {final_stacked_mae:.4f}")
print(f"R-squared: {final_stacked_r2:.4f}")

# Save the final meta-model for future use
joblib.dump(final_meta_model, 'advanced_stacking_meta_model.joblib')

In [ ]:
# 4. Compare the performance of all three models

# Calculate the performance metrics for the individual XGBoost and LightGBM models
# Facilitates a direct comparison with the stacked model.

xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
xgb_mae = mean_absolute_error(y_test, xgb_preds)
xgb_r2 = r2_score(y_test, xgb_preds)

lgbm_rmse = np.sqrt(mean_squared_error(y_test, lgbm_preds))
lgbm_mae = mean_absolute_error(y_test, lgbm_preds)
lgbm_r2 = r2_score(y_test, lgbm_preds)

# Calculate the performance metrics for the final stacking ensemble
final_stacked_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
final_stacked_mae = mean_absolute_error(y_test, final_predictions)
final_stacked_r2 = r2_score(y_test, final_predictions)

# Create a summary DataFrame for easy comparison
performance_summary = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'Advanced Stacking Ensemble'],
    'RMSE': [xgb_rmse, lgbm_rmse, final_stacked_rmse],
    'MAE': [xgb_mae, lgbm_mae, final_stacked_mae],
    'R-squared': [xgb_r2, lgbm_r2, final_stacked_r2]
})

print("\n--- Final Model Performance Summary ---")
print(performance_summary.to_string(index=False))

In [ ]:
# 5. Visualise the Comparison Results

# Create the 'images' directory if it does not exist yet
if not os.path.exists('images'):
    os.makedirs('images')

# Set a style for the plots
plt.style.use('fivethirtyeight')

# Plot the actual values vs the predictions from the best model
# The final stacking ensemble is expected to have the best results.

fig, ax = plt.subplots(figsize=(15, 8))
ax.plot(y_test.index, y_test,
        label='Actual Values', color='brown',
        linewidth=3)
ax.plot(y_test.index, final_predictions,
        label='Stacking Ensemble Predictions', color='coral',
        linewidth=2)
ax.set_title('Actual vs Predicted PJME_MW (Test Set)')
ax.set_xlabel('Date')
ax.set_ylabel('PJME_MW')
ax.legend()
plt.savefig('images/ph6_actual_vs_predicted_final_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Plot the distribution of residuals
# A good model would have residuals centered around zero.

fig, ax = plt.subplots(figsize=(10,6))
residuals = y_test - final_predictions
sns.histplot(residuals, bins=50, kde=True, ax=ax, color='green')
ax.set_title('Distribution of Residuals')
ax.set_xlabel('Residuals (Actual - Predicted)')
ax.set_ylabel('Frequency')
ax.legend()
plt.savefig('images/ph6_residual_distribution_final_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook